# Benchmark Paper Experiment
```BibTeX
@article{theate2021application,
  title={An application of deep reinforcement learning to algorithmic trading},
  author={Th{\'e}ate, Thibaut and Ernst, Damien},
  journal={Expert Systems with Applications},
  volume={173},
  pages={114632},
  year={2021},
  publisher={Elsevier}
}
```

## Setup Notebook

In [1]:
import os
import pandas as pd
import numpy as np
import warnings
import sys
import pickle

from tqdm import tqdm

warnings.filterwarnings("ignore")

In [2]:
%%capture
# Silence the installation

INSTALL_DEPS = True
if INSTALL_DEPS:
    # If Kaggle or Colab, you have to manage these. If local, install requirements.txt
    %pip install yfinance==0.2.43
    # %pip install scipy==1.14.1
    # %pip install statsmodels==0.14.2
    # %pip install tabulate==0.9.0
    # %pip install torch==2.4.1
    # %pip install tqdm==4.66.5
    # %pip install gym==0.26.1
    # %pip install numpy==2.1.1
    # %pip install pandas==2.2.2

IN_KAGGLE = IN_COLAB = False
try:
    # https://www.tensorflow.org/install/pip#windows-wsl2
    import google.colab
    from google.colab import drive

    drive.mount("/content/drive")
    DATA_PATH = "/drive/input/drl-dataset-quant"
    IN_COLAB = True
    print("Colab!")
except:
    IN_COLAB = False
if "KAGGLE_KERNEL_RUN_TYPE" in os.environ and not IN_COLAB:
    print("Running in Kaggle...")
    for dirname, _, filenames in os.walk("/kaggle/input"):
        for filename in filenames:
            print(os.path.join(dirname, filename))
    DATA_PATH = "/kaggle/input/drl-dataset-quant"
    sys.path.insert(1, "/kaggle/usr/lib/drlutil")
    IN_KAGGLE = True
    print("Kaggle!")
elif not IN_COLAB:
    IN_KAGGLE = False
    DATA_PATH = "./data/"
    print("running localhost!")

In [3]:
from drlutil import TradingSimulator, PerformanceEstimator

## Experiment Parameters

In [4]:
simulator = TradingSimulator()
strategy = "TDQN"
stock = "Tesla"

features=['Close', 'Low', 'High', 'Volume']
money=100000.
stateLength = 30
observationSpace = 1 + ((stateLength- 1)*len(features))
actionSpace = 2
bounds = [1, 30]
step = 1
numberOfEpisodes = 100
startingDate = '2012-1-1'
endingDate = '2020-1-1'
splitingDate = '2018-1-1'
percentageCosts = [0, 0.1, 0.2]
transactionCosts = percentageCosts[1]/100

stocks = {
    'Google' : 'GOOGL',
    'Apple' : 'AAPL',
    'S&P 500' : 'SPY',
    'NASDAQ 100' : 'QQQ',
    'Microsoft' : 'MSFT',
    'Sony' : '6758.T',
    'JPMorgan Chase' : 'JPM',
    'HSBC' : 'HSBC',
    'ExxonMobil' : 'XOM',
    'Tesla' : 'TSLA',
    'Toyota' : '7203.T',
    'Coca Cola' : 'KO',
}

# Run Experiments

## Test Selected Stock and Strategy

In [5]:
N_EXPERIMENTS = 20
STOCK_RESULTS = {}
for stock in tqdm(stocks):
    train_results = []
    test_results = []
    for i in tqdm(range(N_EXPERIMENTS)):
        strat, train_env, test_env = simulator.simulateNewStrategy(strategy,
                                                                stock,
                                                                startingDate=startingDate,
                                                                endingDate=endingDate,
                                                                splitingDate=splitingDate,
                                                                verbose=False,
                                                                plotTraining=False,
                                                                rendering=False,
                                                                showPerformance=False,
                                                                saveStrategy=False,
                                                                savePlots=False,
                                                                data_dir=DATA_PATH,
                                                                money=money,
                                                                observationSpace=observationSpace,
                                                                actionSpace=actionSpace,
                                                                stateLength=stateLength,
                                                                bounds=bounds,
                                                                step=step,
                                                                numberOfEpisodes=numberOfEpisodes,
                                                                transactionCosts=transactionCosts,
                                                                features=features)
        analyser = PerformanceEstimator(train_env.data)
        train_results.append(analyser.getComputedPerformance())
        analyser = PerformanceEstimator(test_env.data)
        test_results.append(analyser.getComputedPerformance())
    STOCK_RESULTS[stock] = (train_results, test_results)
    if N_EXPERIMENTS > 0 and len(train_results) > 0:
        with open(f'./data/{stock}_train_results.pkl', 'wb') as f:
            pickle.dump(train_results, f)
        with open(f'./data/{stock}_test_results.pkl', 'wb') as f:
            pickle.dump(test_results, f)


  0%|          | 0/12 [15:01<?, ?it/s]


AttributeError: 'PerformanceEstimator' object has no attribute 'getComputedPerformance'

In [ ]:
import scipy.stats as stats

final_summary_df = pd.DataFrame()
for stock in tqdm(stocks):
    with open(f'./data/{stock}_test_results.pkl', 'rb') as f:
        test_results = pickle.load(f)

    metrics_data = {}
    for df in test_results:
        for _, row in df.iterrows():
            metric = row['Metric']
            value = row['Value']
            if metric not in metrics_data:
                metrics_data[metric] = []
            metrics_data[metric].append(value)
    stock_summary = {}

    for metric, values in metrics_data.items():
        data = pd.Series(values)
        mean_val = round(data.mean(), 2)
        std_val = round(data.std(), 2)
        range_val = (round(mean_val - std_val, 2), round(mean_val + std_val, 2))
        t_stat, p_value = stats.ttest_1samp(data, 0)
        p_value = round(p_value, 2)
        stock_summary[metric] = [mean_val, range_val, p_value]

    stock_df = pd.DataFrame(stock_summary, index=["Mean", "-/+", "P-Value"]).T
    stock_df.columns = [f'{stock} {col}' for col in stock_df.columns]
    if final_summary_df.empty:
        final_summary_df = stock_df
    else:
        final_summary_df = pd.concat([final_summary_df, stock_df], axis=1)

final_summary_df

# Benchmark Experiment Results

| Performance Indicator           | Training  | Testing  | B&H      | S&H      | TF      | MR      | TDQN    |
|----------------------------------|-----------|----------|----------|----------|---------|---------|---------|
| Profit & Loss (P&L)              | 11686926  | -9801    | 29847    | -29847   | 73301   | 8600    | 98      |
| Annualized Return                | 36.25%    | 8.03%    | 24.11%   | -7.38%   | 100.00% | 19.02%  | 12.80%  |
| Annualized Volatility            | 45.32%    | 51.64%   | 53.14%   | 46.11%   | 52.70%  | 58.05%  | 52.09%  |
| Sharpe Ratio                     | 1.983     | 0.162    | 0.508    | -0.154   | 0.987   | 0.358   | 0.261   |
| Sortino Ratio                    | 3.196     | 0.204    | 0.741    | -0.205   | 1.229   | 0.539   | 0.359   |
| Maximum Drawdown                 | 15.31%    | 60.36%   | 52.83%   | 54.09%   | 79.91%  | 65.31%  | 58.95%  |
| Maximum Drawdown Duration        | 47 days   | 95 days  | 205 days | 144 days | 229 days| 159 days| 331 days|
| Profitability                    | 54.82%    | 46.38%   | 100.00%  | 0.00%    | 34.38%  | 67.65%  | 38.18%  |
| Ratio Average Profit/Loss        | 3.082     | 1.112    | ∞        | 0.00     | 0.534   | 0.496   | 1.621   |

Training time: 19:08mins
